<h1> Homework 2 - Data from the Web </h1>

In [95]:
## Importation of everything useful
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
sns.set_context('notebook')

import requests
from bs4 import BeautifulSoup

In [96]:
parameters_page = requests.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247')

In [97]:
parameters_html = parameters_page.text

In [98]:
soup = BeautifulSoup(parameters_html, "html.parser")

In [6]:
#print(soup.prettify())

In [99]:
row_list = []
for parameter_cat_html in soup.find_all('select'):
    for option in parameter_cat_html.contents:
        row_dict = {'category':parameter_cat_html['name'], 'name':option.string, 'value':option['value']}
        row_list.append(row_dict)
parameters = pd.DataFrame(row_list)
parameters.head()

,category,name,value
0,ww_x_UNITE_ACAD,None,null
1,ww_x_UNITE_ACAD,Architecture,942293
2,ww_x_UNITE_ACAD,Chimie et génie chimique,246696
3,ww_x_UNITE_ACAD,Cours de mathématiques spéciales,943282
4,ww_x_UNITE_ACAD,EME (EPFL Middle East),637841336


<h2>Creating the base url</h2>

We found with Postman the base ur
l leading to the students list: http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.bhtml

To this page, we will add different parameters. Some of the parameters will not change for all the request, therefore we decided to hardcode them. These parameters are:
<ul>
<li>
<b>ww_x_GPS</b> = <em>-1</em><br>
This parameter varies during request but is not useful. It is probably linked to the gps position. It is was set to -1 which will accept everything.
</li>
<li>
<b>ww_i_reportModel</b> = <em>133685247</em>:<br>
Parameter to choose the type of files to take. We chose html files.
</li>
<li><b>ww_i_reportModelXsl</b> = <em>133685270</em>:<br>
Parameter to choose the type of files to take. We chose html files.
</li>
<li><b>ww_x_HIVERETE</b> = <em>null</em>:<br>
This parameter allow to choose between Winter and Spring semester. However, it is redondant, as Bachelor 1 can only happen in Winter semester, Bachelor 2 in Spring, Bachelor 3 in Winter, and so forth. Therefore we chose to eliminate this parameter by settings its value to null.
</li>
<li><b>ww_x_UNITE_ACAD</b> = <em>249847</em>:<br>
This value represents the "Informatique" section. As it was asked to only consider the data of this section, we have to use it all the time. If it was not the case, we could use the other technique.
</li>


</ul>

In [100]:
people_base_url = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.bhtml?ww_x_GPS=-1&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=249847&ww_x_HIVERETE=null'

***Interesting parameters***<br>
Therefore we are left with two interesting parameters we want to vary:
the year ("periode academique") and the period ("periode pedagogique").

In [101]:
year_param_string = 'ww_x_PERIODE_ACAD'
period_param_string = 'ww_x_PERIODE_PEDAGO'

year_param = parameters.loc[parameters['category'] == year_param_string]
period_param = parameters.loc[parameters['category'] == period_param_string]

year_param = year_param.drop(year_param.iloc[0].name,axis=0, level=None)
period_param = period_param.drop(period_param.iloc[0].name,axis=0, level=None)
period_param = period_param[period_param['name'] != 'Mise à niveau']

In [102]:
year_param.head()

,category,name,value
21,ww_x_PERIODE_ACAD,2016-2017,355925344
22,ww_x_PERIODE_ACAD,2015-2016,213638028
23,ww_x_PERIODE_ACAD,2014-2015,213637922
24,ww_x_PERIODE_ACAD,2013-2014,213637754
25,ww_x_PERIODE_ACAD,2012-2013,123456101


In [75]:
period_param

,category,name,value
32,ww_x_PERIODE_PEDAGO,Bachelor semestre 1,249108
33,ww_x_PERIODE_PEDAGO,Bachelor semestre 2,249114
34,ww_x_PERIODE_PEDAGO,Bachelor semestre 3,942155
35,ww_x_PERIODE_PEDAGO,Bachelor semestre 4,942163
36,ww_x_PERIODE_PEDAGO,Bachelor semestre 5,942120
37,ww_x_PERIODE_PEDAGO,Bachelor semestre 5b,2226768
38,ww_x_PERIODE_PEDAGO,Bachelor semestre 6,942175
39,ww_x_PERIODE_PEDAGO,Bachelor semestre 6b,2226785
40,ww_x_PERIODE_PEDAGO,Master semestre 1,2230106
41,ww_x_PERIODE_PEDAGO,Master semestre 2,942192


In [103]:
#Construct a beautiful dataframe from a url
def create_df(url):
    
    # pd.read returns a list of dataframe
    # in our case, it returns a list of one dataframe,
    # so we select the first item of this list
    df = pd.read_html(url)[0]
    
    # We change the name of the columns by the value of the second line of each columns
    for i in range(0, len(df.columns)):
        df=df.rename(columns = {i:df.loc[1][i]})
        
    # We add a column for the period
    string = df['Civilité'][0]
    splitedString = string.split(',')
    if False : #splitedString[0] != 'Informatique':
        return 'Not informatic section'
    else :
        df['Period Academic'] = splitedString[1]
        periodPedagogic = splitedString[2].split(' ')[1:4]
        df['Period pedagogic'] = ' '.join(periodPedagogic)

        # We drop the unuseful rows
        # And we drop the columns that contains only NaN
        # TODO: find a way to get the information of the first row (the row [0]), this row describe the year (period academic) and the period pedagogic
        df = df.drop(df.index[[0,1]]).dropna(axis=1, how='all')
        return df

In [104]:
#Creating 2 dataframe for testing
df1 = create_df("http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS=-1&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=944590&ww_x_PERIODE_ACAD=213638028&ww_x_PERIODE_PEDAGO=249108&ww_x_HIVERETE=2936286")
df2 = create_df("http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS=-1&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=944590&ww_x_PERIODE_ACAD=355925344&ww_x_PERIODE_PEDAGO=249108&ww_x_HIVERETE=2936286")

In [105]:
# Let's imagine we have 2 lists of datframe, listOne (of Bachelor 1 for all year) and listSix (Bachelor 6)
listOne = [df1]
listSix = [df2]

In [106]:
def merged_inner(df1, df2):
    merged_inner = pd.merge(left=df1,right=df2, on='No Sciper')
    return merged_inner

In [107]:
# input: 2 dataframes with the correct year in Period Academic (see above)
# Output: a dataframe with the Civilité, the No Sciper and the duration of the Bachelor
def duration_bachelor(df1, df2):
    merged = merged_inner(df1, df2)
    result = pd.DataFrame({'No Sciper' : merged['No Sciper'], 'Civilité' : merged['Civilité_x'], 'Duration of Bachelor' : merged['Period Academic_y'] - merged['Period Academic_x']})
    return result

In [108]:
def correcting_year(listOne, listSix):
    #For all Bachelor 1, we select the firt year of the column Period Academic
    for df in listOne:
        df["Period Academic"]=int(df['Period Academic'][2].split('-')[0].replace(" ", ""))

    #For all Bachelor 6, we select the second year of the column Period Academic
    for df in listSix:
        df["Period Academic"] = int(df['Period Academic'][2].split('-')[1])   

In [109]:
# Iterate for every academic year
# Input : 2 lists, one for Bachelor 1 and one for Bchelor 6
def final_Bachelor(listOne, listSix):
    finalResult = pd.DataFrame()
    
    correcting_year(listOne, listSix)
    for one in listOne:
        for six in listSix:
            
            result = duration_bachelor(one, six)

            # ignore_index=True since the different dataframes that will be append will have the same index
            # verify_integrity=True since we should not have the same student added several time (it makes no sense, a student can finish its Bachelor only one time)
            finalResult = finalResult.append(result,  ignore_index=True, verify_integrity=True)
    #finalResult['Duration of Bachelor']
            
    return finalResult

In [110]:
#Partition the data between male and female students and calculate a mean for particular gender
def mean_Bachelor(df, civilite):
    return df[df['Civilité'] == civilite]['Duration of Bachelor'].mean() 

In [111]:
dfRes = final_Bachelor(listOne, listSix)

In [112]:
#average duration for male
mean_Bachelor(dfRes, 'Monsieur')

2.0

In [113]:
#average duration for female
mean_Bachelor(dfRes, 'Madame')

2.0

*** Make the dataframe and save it on files: ***

In [78]:
def url_param_str(param, value):
    return '&' + param + '=' + value

for i, year in year_param.iterrows():
    for j, period in period_param.iterrows():
            
            url_i = people_base_url + url_param_str(year.category, year.value) + url_param_str(period.category, period.value)
            file_name_i = './data/' + str(year['name']+period['name'])
            print(file_name_i)
#             print(url_i)
            try:
                df_i = create_df(url_i)
            except (ValueError, KeyError):
                print('-------> no file here!')
            else:
                df_i.to_pickle(file_name_i)

test_dataframe = pd.read_pickle("./data/2016-2017Bachelor semestre 1")
test_dataframe.head()

./data/2016-2017Bachelor semestre 1
./data/2016-2017Bachelor semestre 2
./data/2016-2017Bachelor semestre 3
./data/2016-2017Bachelor semestre 4
./data/2016-2017Bachelor semestre 5
./data/2016-2017Bachelor semestre 5b
-------> no file here!
./data/2016-2017Bachelor semestre 6
./data/2016-2017Bachelor semestre 6b
-------> no file here!
./data/2016-2017Master semestre 1
./data/2016-2017Master semestre 2
./data/2016-2017Master semestre 3
./data/2016-2017Master semestre 4
-------> no file here!
./data/2016-2017Mineur semestre 1
-------> no file here!
./data/2016-2017Mineur semestre 2
-------> no file here!
./data/2016-2017Projet Master automne
./data/2016-2017Projet Master printemps
-------> no file here!
./data/2016-2017Semestre automne
./data/2016-2017Semestre printemps
./data/2016-2017Stage automne 3ème année
-------> no file here!
./data/2016-2017Stage automne 4ème année
-------> no file here!
./data/2016-2017Stage printemps 3ème année
-------> no file here!
./data/2016-2017Stage printe

,Civilité,Nom Prénom,Statut,No Sciper,Period Academic,Period pedagogic
2,Monsieur,Abbey Alexandre,Présent,235688,2016-2017,Bachelor semestre 1
3,Monsieur,Ahn Seongho,Présent,274015,2016-2017,Bachelor semestre 1
4,Madame,Alemanno Sara,Présent,268410,2016-2017,Bachelor semestre 1
5,Monsieur,Althaus Luca,Présent,271464,2016-2017,Bachelor semestre 1
6,Monsieur,Assi Karim,Présent,274518,2016-2017,Bachelor semestre 1


In [55]:
test = create_df('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.bhtml?ww_x_GPS=-1&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=249847&ww_x_HIVERETE=null&ww_x_PERIODE_ACAD=355925344&ww_x_PERIODE_PEDAGO=2226768')
test

ValueError: No tables found matching pattern '.+'